# Procesamiento natural del lenguaje
## Carrera de especialización en inteligencia artificial

# Desafío 2: 

## Embeddings de palabras basado en **Martin Fierro**
**Alumno:**

**Nahuel Otonelo Canale**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from gensim.models.callbacks import CallbackAny2Vec

from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

from gensim.models import Word2Vec

import plotly.graph_objects as go
import plotly.express as px
import nltk
from nltk.corpus import stopwords


Para este trabajo creé embeddings de las dos obras de **Jose Hernández**:

- **El Gaucho Martin Fierro**
- **La vuelta de Martin Fierro**

Estos archivos fueron extraidos de la pagina propuesta textos.info. Descargue archivos ePub (.epub) y luego los transforme a txt en una herramienta de conversión online. Luego eliminé manualmente cualquier texto introductorio, prólogos o índices para quedarme únicamente con los versos de la obra.

Para la Tokeneización utilicé la libreria NLTK.




In [ ]:


try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab')

print("--- Recursos de NLTK listos. ---")

# --- Cargar y combinar archivos .txt ---
file_names = [
    'Jose-Hernandez-El-Gaucho-Martin-Fierro.txt', 
    'Jose-Hernandez-La-Vuelta-de-Martin-Fierro.txt'
]

full_text = ""
for filename in file_names:
    print(f"Cargando {filename}...")
    with open(filename, 'r', encoding='utf-8') as f:
        full_text += f.read()
        full_text += "\n"

print(f"\n¡Corpus cargado! Total de caracteres: {len(full_text)}")


# --- Convertir el poema en "Prosa" (un solo texto largo) ---
# esto lo hago asi porque los versos son de muy pocas palabras...

prosa_completa = full_text.replace('\n', ' ').strip()
print("Poema convertido en una sola línea de prosa.")


# --- Tokenizar la prosa completa (CON Stop Words) ---
print("Tokenizando el texto completo...")

# 1. Tokenizamos con NLTK
tokens_originales = nltk.word_tokenize(
    prosa_completa, 
    language='spanish', 
    preserve_line=True 
)

# 2. Filtramos SOLO puntuación y números
tokens_totales = []
for palabra in tokens_originales:
    palabra_lower = palabra.lower()
    
    # Este filtro MANTIENE las stop words (ej. 'la', 'que')
    if (palabra.isalpha()): 
        tokens_totales.append(palabra_lower)

print(f"Tokenización completa. Total de palabras en el corpus: {len(tokens_totales)}")


# --- Crear 'sentence_tokens' (dividir el texto largo) ---
# Dividimos la lista gigante de palabras en trozos (documentos)
chunk_size = 100  # Documentos de 100 palabras cada uno
sentence_tokens = []

for i in range(0, len(tokens_totales), chunk_size):
    chunk = tokens_totales[i : i + chunk_size]
    if chunk:
        sentence_tokens.append(chunk)

# --- 6. Verificar el resultado ---
print(f"Corpus dividido en {len(sentence_tokens)} documentos de ~{chunk_size} palabras.")
print("\nEjemplo del primer documento (CON stop words):")
print(sentence_tokens[0])

--- Recursos de NLTK listos. ---
Cargando Jose-Hernandez-El-Gaucho-Martin-Fierro.txt...
Cargando Jose-Hernandez-La-Vuelta-de-Martin-Fierro.txt...

¡Corpus cargado! Total de caracteres: 196014
Poema convertido en una sola línea de prosa.
Tokenizando el texto completo...
Tokenización completa. Total de palabras en el corpus: 32665
Corpus dividido en 327 documentos de ~100 palabras.

Ejemplo del primer documento (CON stop words):
['i', 'aquí', 'me', 'pongo', 'a', 'cantar', 'al', 'compás', 'de', 'la', 'vigüela', 'que', 'el', 'hombre', 'que', 'lo', 'desvela', 'una', 'pena', 'estrordinaria', 'como', 'la', 'ave', 'solitaria', 'con', 'el', 'cantar', 'se', 'pido', 'a', 'los', 'santos', 'del', 'cielo', 'que', 'ayuden', 'mi', 'pensamiento', 'les', 'pido', 'en', 'este', 'momento', 'que', 'voy', 'a', 'cantar', 'mi', 'historia', 'me', 'refresquen', 'la', 'memoria', 'y', 'aclaren', 'mi', 'vengan', 'santos', 'milagrosos', 'vengan', 'todos', 'en', 'mi', 'ayuda', 'que', 'la', 'lengua', 'se', 'me', 'añud

In [3]:
#Uso esta clase del ejemplo del notebook de la materia NLP

# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# Crearmos el modelo generador de vectores con NUEVOS PARÁMETROS
w2v_model = Word2Vec(min_count=3,    # BAJAMOS: para incluir más vocabulario gauchesco
                     window=5,       # SUBIMOS: para capturar el contexto de la prosa (sopbre todo teniendo en cuenta que tenemos texto en prosa)
                     vector_size=100,  
                     negative=20,    
                     workers=1,      
                     sg=1)           # Skip-gram

print("Modelo definido con parámetros optimizados.")

Modelo definido con parámetros optimizados.


In [5]:
w2v_model.build_vocab(sentence_tokens)

In [6]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=50,  # <-- SUBIMOS A 50 ÉPOCAS (para que aprenda más con un corpus chico)
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 500409.84375
Loss after epoch 1: 386959.59375
Loss after epoch 2: 360269.1875
Loss after epoch 3: 343245.25
Loss after epoch 4: 345659.5
Loss after epoch 5: 321257.125
Loss after epoch 6: 300583.0
Loss after epoch 7: 298581.5
Loss after epoch 8: 296622.75
Loss after epoch 9: 292943.75
Loss after epoch 10: 294034.25
Loss after epoch 11: 291887.0
Loss after epoch 12: 280869.75
Loss after epoch 13: 271941.5
Loss after epoch 14: 272333.5
Loss after epoch 15: 270500.5
Loss after epoch 16: 266753.5
Loss after epoch 17: 267280.0
Loss after epoch 18: 264980.5
Loss after epoch 19: 264145.5
Loss after epoch 20: 262494.5
Loss after epoch 21: 260612.0
Loss after epoch 22: 262524.0
Loss after epoch 23: 259018.5
Loss after epoch 24: 259985.5
Loss after epoch 25: 259575.0
Loss after epoch 26: 257723.5
Loss after epoch 27: 257684.0
Loss after epoch 28: 248074.5
Loss after epoch 29: 241718.0
Loss after epoch 30: 240765.0
Loss after epoch 31: 241846.0
Loss after epoch 32: 242221.0
Lo

(848700, 1633250)

In [7]:
print("Cantidad de words distintas en MI corpus (Martín Fierro):", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en MI corpus (Martín Fierro): 1359


## Palabras que mas se relacionan con...

Observemos los algunos términos interesantes y buscamos términos más similares y menos similares.

In [8]:
# --- lista de palabras de interés ---
palabras_de_interes = ["vigüela", "gaucho", "pena", "facón", "muerte", "baile", "caballo", "amor", "desierto", "cantar", "guitarra", "luna", "sol"]

print("--- ANÁLISIS DE SIMILITUD ---")

# --- Inicia el loop ---
for palabra in palabras_de_interes:
    
    print(f"\n--- {palabra.upper()} ---") # Título dinámico
    
    try:
        # Intenta buscar la palabra
        similares = w2v_model.wv.most_similar(positive=[palabra], topn=10)
        print(similares)
        
    except KeyError:
        # Si da error (como "vigüela"), avisa en lugar de fallar
        print(f"ERROR: La palabra '{palabra}' NO se encuentra en el vocabulario.")

--- ANÁLISIS DE SIMILITUD ---

--- VIGÜELA ---
ERROR: La palabra 'vigüela' NO se encuentra en el vocabulario.

--- GAUCHO ---
[('necesita', 0.4836764335632324), ('palo', 0.47907692193984985), ('dotor', 0.43993693590164185), ('manso', 0.43778786063194275), ('aguanta', 0.42998939752578735), ('marido', 0.42678824067115784), ('malo', 0.425479531288147), ('defiende', 0.42052820324897766), ('canta', 0.4177068769931793), ('jesús', 0.41347506642341614)]

--- PENA ---
[('agena', 0.6202473044395447), ('llena', 0.6116396188735962), ('nueva', 0.6052500009536743), ('crece', 0.5891825556755066), ('ave', 0.5723959803581238), ('tormentos', 0.5701315402984619), ('apriende', 0.5591505765914917), ('situación', 0.5539376735687256), ('alma', 0.5513513088226318), ('bendita', 0.5460655093193054)]

--- FACÓN ---
[('punta', 0.6933119893074036), ('monté', 0.68050217628479), ('dijunto', 0.6725101470947266), ('ganas', 0.6418548822402954), ('respeto', 0.6278272867202759), ('carreta', 0.6054339408874512), ('oír', 0

No en todas las palabras hay buenas similitudes.

Por algun error de lectura del archivo, vigüela no aparece en el diccionario...

- Para **Facón** hay algunas como punta, respeto, maldita.
- **Muerte**: aguantar, gallina (¿de miedoso?), crueldá 
- **Baile** se relaciona con milonga, negra, alaridos
- **Caballo**: pampa, rayo (¿el caballo corre como un rayo?), correr, pingo (asi le dicen a los caballos), ama (amor a los caballos)
- **Amor**: guerra, volverme, marido, manso
- **Desierto**: cruzar, dejamos, irse, dejando, tristeza
- **Cantar** pájaros, pongo, cantores, fama, gusto, gloria


## Palabras que menos se relacionan con:

In [9]:
# --- Define tu lista de palabras de interés ---
palabras_de_interes = ["vigüela", "gaucho", "pena", "facón", "muerte", "baile", "caballo", "amor", "desierto", "cantar", "guitarra", "luna", "sol"]

print("--- ANÁLISIS DE SIMILITUD ---")

# --- Inicia el loop ---
for palabra in palabras_de_interes:
    
    print(f"\n--- {palabra.upper()} ---") # Título dinámico
    
    try:
        # Intenta buscar la palabra
        similares = w2v_model.wv.most_similar(negative=[palabra], topn=10)
        print(similares)
        
    except KeyError:
        # Si da error (como "vigüela"), avisa en lugar de fallar
        print(f"ERROR: La palabra '{palabra}' NO se encuentra en el vocabulario.")

--- ANÁLISIS DE SIMILITUD ---

--- VIGÜELA ---
ERROR: La palabra 'vigüela' NO se encuentra en el vocabulario.

--- GAUCHO ---
[('prudencia', 0.022662337869405746), ('decir', 0.01990785077214241), ('pido', 0.019257189705967903), ('campo', 0.01913454569876194), ('andar', 0.016486749053001404), ('pusieron', 0.012131823226809502), ('quedó', 0.004309517797082663), ('estao', 0.0010197595693171024), ('muchas', -0.006690514739602804), ('tenía', -0.008431530557572842)]

--- PENA ---
[('largan', 0.04269202798604965), ('daba', -0.009184958413243294), ('moro', -0.010023566894233227), ('ansí', -0.015308280475437641), ('caso', -0.01879044808447361), ('deja', -0.019145837053656578), ('defender', -0.03661595284938812), ('andar', -0.0373169407248497), ('pronto', -0.04266753047704697), ('frontera', -0.04949672520160675)]

--- FACÓN ---
[('tiene', 0.011058522388339043), ('palabra', -0.03772055730223656), ('compasión', -0.038745395839214325), ('anda', -0.0539478175342083), ('cueva', -0.054411761462688446)

Mencionando algunas particularidades:

- **Gaucho** prudencia, campo (¿campo? es raro...)




Para poder ver mejor las semejanzas de palabras, realizamos un gráfico de dos dimensiones. Para ello primero debemos realizar una disminucion de dimensionalidad.

In [10]:
def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [11]:
# Graficar los embedddings en 2D

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=800
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show()

## Encontrando familiaridad entre las palabras:

Hace ya varios años que leí Martin Fierro en la escuela primaria, y aunque me cueste un poco, trataré de encontrar similitudes en las palabras. ¿Por qué algunas palabras estan cerca de otras en el gráfico?


- [cuatro vientos]: del dicho a cuatro vientos

![cuatrovientos.png](./cuatrovientos.png)


- tandanza, tiempo: 

![tardanzatiempo.png](./tardanzatiempo.png)

y de la misma imágen "canta" y "canto" mas a la derecha. A la izquierda, casi sobre el eje y, "sufrido", "sufrir" y "llanto".



- stop words: pueden verse juntas algunas.

![stopwords.png](./stopwords.png)


- soleda, sufre, pena, solo, muger, amor. El gaucho era un ser solitario, talvez sufria su soledad.
![soledasufre.png](./soledasufre.png)

